In [9]:
from tensorflow.keras.preprocessing import image
import os
import pandas as pd
import numpy as np
from skimage.measure import block_reduce
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
import tensorflow as tf

In [4]:
df = pd.read_csv('drive/MyDrive/cleaned_4500.csv')

df['spectogram'] = [f'{f}.jpg' if os.path.isfile(f'drive/MyDrive/specs/{f}.jpg') else 'None' for f in range(df.shape[0])]
ndf = df.iloc[:,[-2,-1]]
ndf = ndf[ndf['spectogram'] != 'None']
ndf.reset_index(drop=True, inplace=True)

In [5]:
def normalize_gray(array):
    return (array - array.min())/(array.max() - array.min())


Y, X = np.split(ndf.values, 2, axis=1)

Y = np.hstack(Y)
X = np.hstack(X)
img_ar_gray = np.array([image.img_to_array(image.load_img('drive/MyDrive/specs/' + fname, target_size=(288, 432)))[...,:3]
                        @ [0.299, 0.587, 0.114] for fname in X])
ar = normalize_gray(img_ar_gray)

In [6]:
X = np.array([block_reduce(img, block_size = (3,3), func=np.mean) for img in ar])
Y = pd.get_dummies(Y).values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

X_train = tf.expand_dims(X_train, axis=-1)
X_test = tf.expand_dims(X_test, axis=-1)

In [7]:
print(X_test.shape)
input_shape = (96, 144, 1)
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(17, activation='softmax'))
model.compile(optimizer='adam',
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30)
preds = model.predict(X_test, batch_size=100)

(196, 96, 144, 1)
Epoch 1/30
25/25 [==============================] - 12s 43ms/step - loss: 2.5455 - accuracy: 0.4474
Epoch 2/30
25/25 [==============================] - 1s 31ms/step - loss: 1.6222 - accuracy: 0.4987
Epoch 3/30
25/25 [==============================] - 1s 31ms/step - loss: 1.6199 - accuracy: 0.5205
Epoch 4/30
25/25 [==============================] - 1s 30ms/step - loss: 1.4342 - accuracy: 0.5474
Epoch 5/30
25/25 [==============================] - 1s 31ms/step - loss: 1.4088 - accuracy: 0.5474
Epoch 6/30
25/25 [==============================] - 1s 31ms/step - loss: 1.2796 - accuracy: 0.5744
Epoch 7/30
25/25 [==============================] - 1s 31ms/step - loss: 1.1396 - accuracy: 0.6256
Epoch 8/30
25/25 [==============================] - 1s 31ms/step - loss: 0.9506 - accuracy: 0.6910
Epoch 9/30
25/25 [==============================] - 1s 31ms/step - loss: 0.8311 - accuracy: 0.7090
Epoch 10/30
25/25 [==============================] - 1s 31ms/step - loss: 0.6620 - accurac

In [10]:
th, _ = np.split(ndf.values, 2, axis=1)
th = np.hstack(th)
dummies = pd.get_dummies(th)

d_c = dummies.copy()
d_c = d_c.iloc[:196]
d_c[dummies.columns] = preds
s1 = d_c.idxmax(axis=1).values

d_c = dummies.copy()
d_c = d_c.iloc[:196]
d_c[dummies.columns] = y_test
s2 = d_c.idxmax(axis=1).values

inds = s2 != s1
heyo = np.dstack((s1[inds],s2[inds]))
print(heyo.shape)

(1, 58, 2)
